# Carbon Intensity Forecasts

This notebook demonstrates some ways that this carbon intensity forecasting package can be used.

The first draft of this notebook was completed by Julian de Hoog in March 2021.  Much of this work builds on previous analyses by Maneesha Perera.

---

### Imports etc.

In [1]:
# Standard libraries
import pandas as pd 
from datetime import datetime, timedelta
import os
import pickle
import time

# Forecasting models
from sktime.forecasting.model_selection import temporal_train_test_split

# Plotting with plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Local imports
from ciforecast import CarbonIntensityForecastModels
from ciforecast import generate_forecast, generate_forecast_from_ci, generate_forecast_from_mix
from ciforecast.util import plots as plots
from ciforecast.util.util import calculate_error

# Enable modules to be reloaded
%load_ext autoreload
%autoreload 2

---

## 1. Import data

We can either (i) import data directly from quasar, or (ii) load it from a file we previously created.

---

### Load data directly from quasar

Only run the blocks in this section if retrieving data directly from quasar -- you will need to have `quasarclient` installed.

In [1]:
from quasarclient import QuasarClient

In [2]:
# Create quasar client
with open("credentials/quasar_api_key") as file:
    file_contents = file.readlines()
    quasar_api_token = file_contents[0]
client = QuasarClient(quasar_api_token=quasar_api_token)

Let's use data from Germany as an example -- let's use January-February 2021 for now.

In [7]:
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 3, 1)

In [4]:
data = client.get_region_production(
    'DE', 
    start_date=start_date, 
    end_date=end_date,
    include_carbon_intensity=True
)

NameError: name 'region' is not defined

Let's have a quick look at data

In [9]:
data

,coal,nuclear,gas,geothermal,wind,oil,mixed,hydro,solar,biomass,carbon_intensity
timestamp,,,,,,,,,,,
2021-01-01 00:00:00+00:00,14784,8150,5451,5,4190,406,671,1241,0,5087,419.758788
2021-01-01 00:15:00+00:00,14628,8146,5393,5,4099,406,671,1188,0,5072,419.658435
2021-01-01 00:30:00+00:00,14569,8153,5390,5,3938,404,671,1178,0,5089,420.660719
2021-01-01 00:45:00+00:00,14609,8152,5402,5,3800,404,671,1178,0,5098,422.522718
2021-01-01 01:00:00+00:00,14738,8156,5343,5,3690,406,680,1174,0,5092,424.972050
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-26 00:45:00+00:00,12175,8062,7248,24,10126,388,539,1521,0,5057,342.934531
2021-02-26 01:00:00+00:00,11959,8065,7215,24,10286,387,539,1541,0,5052,339.155292
2021-02-26 01:15:00+00:00,11622,8066,7173,24,10484,388,540,1533,0,5078,333.895961


And let's plot it too.  There is a dedicated package with plots for this purpose -- will need to be installed.

In [23]:
iplot(plots.plot_energy_mix_and_carbon_intensity(data))

There do appear to be some gaps in this data (e.g. Jan 7-8), so let's resample and interpolate to ensure there are no issues when exploring forecasts.  While we're at it, let's convert to hourly data as well.

In [116]:
len(data)

5384

In [117]:
data = data.resample('1h').mean().interpolate('linear')

In [118]:
len(data) 

1346

Finally, let's store this data to a local file for easy retrieval later.

In [119]:
# Choose path and filename
pkl_path = os.path.join('data/example_data_germany.pickle')

# Create directories if needed
if not os.path.exists(os.path.dirname(pkl_path)):
    os.makedirs(os.path.dirname(pkl_path))
    
# Write file
with open(pkl_path, 'wb') as file:
    pickle.dump(data, file)

---

### Import data from local file

If a data file was provided or previously generated, read data in from file

In [2]:
pkl_path = os.path.join('data/example_data_germany.pickle')

with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [3]:
# Let's quickly check data
data

,coal,nuclear,gas,geothermal,wind,oil,mixed,hydro,solar,biomass,carbon_intensity
timestamp,,,,,,,,,,,
2021-01-01 00:00:00+00:00,14647.50,8150.25,5409.00,5.0,4006.75,405.00,671.00,1196.25,0.0,5086.50,420.650165
2021-01-01 01:00:00+00:00,14825.75,8156.50,5312.00,5.0,3573.75,405.50,680.00,1164.00,0.0,5077.50,427.235780
2021-01-01 02:00:00+00:00,15190.00,8153.75,5127.00,5.0,3145.25,405.50,680.25,1179.25,0.0,5061.00,435.282478
2021-01-01 03:00:00+00:00,15108.25,8150.50,4993.00,5.0,2828.50,404.75,695.75,1155.00,0.0,5045.25,438.189120
2021-01-01 04:00:00+00:00,15436.00,8150.25,5057.50,5.0,2608.75,405.00,697.25,1145.50,0.0,5064.00,444.054551
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-25 21:00:00+00:00,14479.50,8040.00,8239.75,24.0,10367.75,387.00,543.75,1504.75,0.0,5097.25,367.288315
2021-02-25 22:00:00+00:00,13851.50,8049.50,7781.25,24.0,10079.00,387.50,543.00,1491.25,0.0,5076.50,362.273912
2021-02-25 23:00:00+00:00,12834.50,8055.00,7467.00,24.0,9797.50,387.50,539.00,1495.75,0.0,5080.00,353.180841


---

## 2. Simple data-driven models - carbon intensity only

Let's initially explore a few simple data-driven models on the carbon intensity time series only.

Since we are assuming day-ahead forecasts for now, let's hold out last day of data for comparison.

In [4]:
train_end = data.index[-1] - timedelta(days=1)
test_start = train_end + timedelta(hours=1)
train = data.loc[:train_end, 'carbon_intensity']
test = data.loc[test_start:, 'carbon_intensity']

Now let's remind ourselves of available forecasting models

In [5]:
for model in CarbonIntensityForecastModels:
    print(model)

CarbonIntensityForecastModels.SEASONAL_NAIVE
CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
CarbonIntensityForecastModels.AUTO_ETS
CarbonIntensityForecastModels.ARIMA
CarbonIntensityForecastModels.AUTO_ARIMA
CarbonIntensityForecastModels.PROPHET
CarbonIntensityForecastModels.PERIODIC_PERSISTENCE


Now let's write a simple function to generate and display a forecast.

In [6]:
def generate_and_display(model, params=None):
    start_time = time.time()
    forecast = generate_forecast(train, model, params=params)
    iplot(plots.plot_single_time_series_with_fc(train, forecast, actual=test, 
                                                show_from=train.index[-1]-timedelta(days=3)))
    print("   Error: %.4f" % calculate_error(train, test, forecast))
    print("Run time: %.4f seconds" % float(time.time()-start_time))

And now let's explore a few simple models.

In [7]:
generate_and_display(CarbonIntensityForecastModels.SEASONAL_NAIVE)

AttributeError: 'Int64Index' object has no attribute 'tz_localize'

In [8]:
generate_and_display(CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING)

AttributeError: 'Int64Index' object has no attribute 'tz_localize'

In [57]:
generate_and_display(CarbonIntensityForecastModels.AUTO_ETS)

   Error: 0.8492
Run time: 6.6588 seconds


In [58]:
generate_and_display(CarbonIntensityForecastModels.AUTO_ARIMA)

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning:

As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.



   Error: 1.1510
Run time: 128.5955 seconds


In [59]:
generate_and_display(CarbonIntensityForecastModels.ARIMA)

   Error: 1.1171
Run time: 0.5604 seconds


In [60]:
generate_and_display(CarbonIntensityForecastModels.PROPHET)

   Error: 0.5618
Run time: 11.4391 seconds


In [61]:
generate_and_display(CarbonIntensityForecastModels.PERIODIC_PERSISTENCE)

   Error: 0.6755
Run time: 0.0486 seconds


---

### More thorough evaluation

So far we've only explored performance on a single day.  Let's do a more thorough evaluation by averaging for all forecast models across a longer testing period.

In [16]:
# Number of days to test
NUM_TEST_DAYS = 7

In [17]:
# Initialise tracking variables
error = {}
runtime = {}
for model in CarbonIntensityForecastModels:
    error[model] = 0
    runtime[model] = 0

In [18]:
# Initialise first training data
train_end = data.index[-1] - timedelta(days=NUM_TEST_DAYS)
train = data.loc[:train_end, 'carbon_intensity']

In [19]:
# Loop through all days in testing
for day in range(0, NUM_TEST_DAYS):
    
    print(day+1, "of", NUM_TEST_DAYS)
    
    # Create test data
    test_start = train.index[-1] + timedelta(hours=1)
    test_end = test_start + timedelta(days=1) - timedelta(hours=1)
    test = data.loc[test_start:test_end, 'carbon_intensity']
    
    # Loop through models
    for model in CarbonIntensityForecastModels:
        
        print(" -", model)
    
        # Train model & update tracking variables
        start_time = time.time()
        forecast = generate_forecast(train, model)
        runtime[model] = runtime[model] + float(time.time()-start_time)
        error[model] = error[model] + calculate_error(train, test, forecast)
    
    # Update train data set
    train = train.append(test)

1 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
 - CarbonIntensityForecastModels.AUTO_ETS
 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.AUTO_ARIMA


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning:

As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.



 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
 - CarbonIntensityForecastModels.PROPHET
2 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
 - CarbonIntensityForecastModels.AUTO_ETS
 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.AUTO_ARIMA


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning:

As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.



 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
 - CarbonIntensityForecastModels.PROPHET
3 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
 - CarbonIntensityForecastModels.AUTO_ETS
 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.AUTO_ARIMA


...

 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
 - CarbonIntensityForecastModels.PROPHET
6 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
 - CarbonIntensityForecastModels.AUTO_ETS
 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.AUTO_ARIMA


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning:

As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.



 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
 - CarbonIntensityForecastModels.PROPHET
7 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING
 - CarbonIntensityForecastModels.AUTO_ETS
 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.AUTO_ARIMA


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pmdarima/arima/arima.py:577: UserWarning:

As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.



 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
 - CarbonIntensityForecastModels.PROPHET


In [26]:
print(error)
# Also write to file
pkl_path = os.path.join('data/error_CI_only.pickle')
with open(pkl_path, 'wb') as file:
    pickle.dump(data, file)

{<CarbonIntensityForecastModels.SEASONAL_NAIVE: 'seasonal_naive'>: 6.48094481413668, <CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING: 'exponential_smoothing'>: 4.305302452232377, <CarbonIntensityForecastModels.AUTO_ETS: 'auto_ets'>: 4107.554507737989, <CarbonIntensityForecastModels.ARIMA: 'arima'>: 6.462819683798054, <CarbonIntensityForecastModels.AUTO_ARIMA: 'auto_arima'>: 4.957601639845063, <CarbonIntensityForecastModels.PERIODIC_PERSISTENCE: 'periodic_persistence'>: 5.451011209701551, <CarbonIntensityForecastModels.PROPHET: 'prophet'>: 7.896360247843908}


In [25]:
print(runtime)
# Also write to file
pkl_path = os.path.join('data/runtime_CI_only.pickle')
with open(pkl_path, 'wb') as file:
    pickle.dump(data, file)

{<CarbonIntensityForecastModels.SEASONAL_NAIVE: 'seasonal_naive'>: 0.1104888916015625, <CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING: 'exponential_smoothing'>: 2.2277913093566895, <CarbonIntensityForecastModels.AUTO_ETS: 'auto_ets'>: 27.52250361442566, <CarbonIntensityForecastModels.ARIMA: 'arima'>: 2.8670310974121094, <CarbonIntensityForecastModels.AUTO_ARIMA: 'auto_arima'>: 1905.2774243354797, <CarbonIntensityForecastModels.PERIODIC_PERSISTENCE: 'periodic_persistence'>: 0.020649433135986328, <CarbonIntensityForecastModels.PROPHET: 'prophet'>: 74.12983202934265}


In [27]:
error2 = {}
for m in error:
    error2[m.value] = error[m]/NUM_TEST_DAYS
error2

{'seasonal_naive': 0.9258492591623828,
 'exponential_smoothing': 0.6150432074617681,
 'auto_ets': 586.793501105427,
 'arima': 0.9232599548282935,
 'auto_arima': 0.7082288056921519,
 'periodic_persistence': 0.7787158871002215,
 'prophet': 1.128051463977701}

In [28]:
runtime2 = {}
for m in error:
    runtime2[m.value] = runtime[m]/NUM_TEST_DAYS
runtime2

{'seasonal_naive': 0.015784127371651784,
 'exponential_smoothing': 0.3182559013366699,
 'auto_ets': 3.931786230632237,
 'arima': 0.4095758710588728,
 'auto_arima': 272.1824891907828,
 'periodic_persistence': 0.0029499190194266184,
 'prophet': 10.589976004191808}

In [33]:
list(error2.keys())

['seasonal_naive',
 'exponential_smoothing',
 'auto_ets',
 'arima',
 'auto_arima',
 'periodic_persistence',
 'prophet']

In [40]:
trace = go.Scatter(
    x = list(error2.values()),
    y = list(runtime2.values()),
    mode = 'markers+text',
    text = list(error2.keys()),
    textposition = "bottom right",
)
fig = go.Figure(data=[trace])
fig.update_xaxes(title="Average error (MASE)", type="log")
fig.update_yaxes(title="Average running time (seconds)", type="log")
iplot(fig)

---

## 3. Simple data-driven models -- each component of energy mix

In [4]:
# Let's reset our training and testing data
train_end = data.index[-1] - timedelta(days=1)
test_start = train_end + timedelta(hours=1)
train = data.loc[:train_end, :]
test = data.loc[test_start:, :]

In [46]:
def generate_and_display_with_energy_mix(model, params=None):
    start_time = time.time()
    forecast = generate_forecast(train, model, params=params)
    iplot(plots.plot_full_mix_with_fc(train, forecast, actual=test, show_from=train.index[-1]-timedelta(days=3)))
    print("   Error: %.4f" % calculate_error(train['carbon_intensity'], 
                                             test['carbon_intensity'], 
                                             forecast['carbon_intensity']))
    print("Run time: %.4f seconds" % float(time.time()-start_time))

In [47]:
generate_and_display_with_energy_mix(CarbonIntensityForecastModels.SEASONAL_NAIVE)

   Error: 0.7764
Run time: 0.3746 seconds


In [49]:
generate_and_display_with_energy_mix(CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING)

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



   Error: 0.5489
Run time: 2.7818 seconds


,coal,nuclear,gas,geothermal,wind,oil,hydro,solar,biomass,carbon_intensity
2021-02-25 02:00:00+00:00,8353.413791,7769.920599,3897.720566,23.985332,22219.731679,393.957102,1477.265520,-2.142748,5052.727552,216.698487
2021-02-25 03:00:00+00:00,8575.950584,7788.849962,4065.322951,23.985154,21608.165469,394.287695,1490.285822,-15.150485,5035.391942,222.854660
2021-02-25 04:00:00+00:00,8406.200275,7789.231194,4379.715920,23.970359,21129.445476,394.367796,1477.175222,-43.793710,5023.097832,224.738317
2021-02-25 05:00:00+00:00,8677.126350,7799.391053,4988.442989,23.955607,20764.911936,394.542008,1481.139528,-76.172252,5035.207894,232.961239
2021-02-25 06:00:00+00:00,9752.492307,7794.143476,5924.291635,23.940914,20614.771762,394.560618,1470.434322,-107.291731,5023.443047,250.931118
2021-02-25 07:00:00+00:00,10690.832375,7803.713031,6896.706515,23.926261,20118.742203,393.479600,1472.550709,-112.960202,5035.089096,267.801772
2021-02-25 08:00:00+00:00,11409.338155,7813.989489,7591.030118,23.911659,19536.431489,392.375105,1478.305342,118.631514,5051.860009,279.828237
2021-02-25 09:00:00+00:00,11901.498967,7836.448220,8093.928667,23.897100,18989.184622,395.048053,1488.937889,487.150575,5037.716691,287.643126
2021-02-25 10:00:00+00:00,12256.829714,7849.167533,8437.241571,23.882571,18549.249706,401.364715,1478.160864,936.443254,5051.841636,292.583694
2021-02-25 11:00:00+00:00,12630.792548,7847.191170,8703.914389,23.868074,18086.238971,401.988180,1480.266732,1386.548858,5065.588234,297.370843


In [50]:
generate_and_display_with_energy_mix(CarbonIntensityForecastModels.ARIMA)

   Error: 1.1927
Run time: 3.7069 seconds


,coal,nuclear,gas,geothermal,wind,oil,hydro,solar,biomass,carbon_intensity
2021-02-25 02:00:00+00:00,8728.427550,7558.042191,3734.524558,23.999997,18032.976299,393.012610,1431.155305,2.223783e-07,5041.949874,240.992135
2021-02-25 03:00:00+00:00,8744.745755,7731.644268,3854.593513,23.999997,24286.531123,393.605921,1472.259265,2.223783e-07,5049.245171,212.980968
2021-02-25 04:00:00+00:00,9061.898568,8085.136773,3877.192719,23.999995,25468.246470,393.363567,1515.772572,6.037791e-07,5016.004007,210.748364
2021-02-25 05:00:00+00:00,9911.042554,8275.595141,4245.061417,23.999992,26927.176599,393.356217,1538.153651,1.098896e-06,4997.253106,215.844681
2021-02-25 06:00:00+00:00,12076.715896,8273.436413,5498.874758,23.999990,28555.507503,393.346580,1509.834115,2.250002e+00,5015.472039,237.444437
2021-02-25 07:00:00+00:00,13977.962014,8214.397828,6063.460679,23.999987,30024.349463,393.336593,1472.338649,8.115000e+02,5029.937467,249.268482
2021-02-25 08:00:00+00:00,14461.982995,8089.480117,6259.103513,23.999985,30246.639827,393.326554,1452.381902,5.466250e+03,5036.152489,239.959426
2021-02-25 09:00:00+00:00,14132.209814,8088.601858,6233.968264,23.999982,29284.295835,393.316506,1447.666634,1.248675e+04,5030.367463,222.153225
2021-02-25 10:00:00+00:00,12841.642464,8065.903137,5941.721440,23.999980,27203.461717,393.306457,1448.703242,1.892775e+04,5054.832432,202.459403
2021-02-25 11:00:00+00:00,11968.482789,8039.430833,5650.679850,23.999977,26574.426516,393.296407,1452.489436,2.288950e+04,5076.547400,188.599308


In [51]:
generate_and_display_with_energy_mix(CarbonIntensityForecastModels.PROPHET)

   Error: 0.3929
Run time: 108.6978 seconds


,coal,nuclear,gas,geothermal,wind,oil,hydro,solar,biomass,carbon_intensity
2021-02-25 02:00:00+00:00,7135.264218,7709.960156,3018.046117,24.299632,15949.699552,378.490663,1466.909345,783.306319,4994.626955,220.949547
2021-02-25 03:00:00+00:00,7076.819499,7709.816372,2965.348248,24.432619,15268.726221,377.262721,1458.718069,994.596626,4972.566889,222.320776
2021-02-25 04:00:00+00:00,7589.420721,7711.327383,3170.948986,24.484993,14741.860048,377.640357,1455.386932,740.633233,4960.603162,235.050591
2021-02-25 05:00:00+00:00,8657.581090,7708.163660,3657.043033,24.513871,14438.483616,378.328905,1456.378433,273.024429,4964.838068,256.904951
2021-02-25 06:00:00+00:00,10018.009258,7703.020837,4325.118234,24.568714,14161.736079,377.770008,1457.941001,405.012938,4983.428470,279.024063
2021-02-25 07:00:00+00:00,11269.596449,7703.439593,4982.849520,24.654256,13614.558097,376.358180,1457.267971,2229.260036,5008.167002,290.295020
2021-02-25 08:00:00+00:00,12088.030391,7712.866318,5447.733359,24.738970,12674.191429,376.630426,1454.719101,6531.157865,5031.398088,284.702919
2021-02-25 09:00:00+00:00,12383.830404,7725.792949,5647.101408,24.794496,11510.323332,380.904690,1452.112716,13160.540101,5051.842642,265.522806
2021-02-25 10:00:00+00:00,12295.834031,7731.656105,5637.842451,24.824374,10434.029068,388.576782,1449.573802,20747.567630,5073.997134,242.354252
2021-02-25 11:00:00+00:00,12053.007450,7723.606094,5540.446407,24.855610,9628.475954,395.895142,1444.958526,27048.282313,5102.062788,224.375735


In [52]:
generate_and_display_with_energy_mix(CarbonIntensityForecastModels.PERIODIC_PERSISTENCE)

   Error: 0.6743
Run time: 0.3141 seconds


,coal,nuclear,gas,geothermal,wind,oil,hydro,solar,biomass,carbon_intensity
2021-02-25 02:00:00+00:00,9271.541667,7792.166667,4788.125000,24.0,18200.041667,395.375000,1473.666667,NaN,4634.083333,251.333024
2021-02-25 03:00:00+00:00,9564.500000,7959.416667,4910.458333,24.0,17771.625000,395.083333,1467.708333,NaN,4612.083333,256.903093
2021-02-25 04:00:00+00:00,10438.041667,8050.833333,5432.625000,24.0,17699.625000,394.958333,1499.125000,NaN,4682.666667,269.337064
2021-02-25 05:00:00+00:00,12703.541667,8048.250000,6475.250000,24.0,18444.166667,394.916667,1468.375000,1.583333,4689.958333,294.086512
2021-02-25 06:00:00+00:00,14034.000000,8020.166667,7061.958333,24.0,19008.708333,394.708333,1461.791667,720.333333,4724.041667,302.753883
2021-02-25 07:00:00+00:00,14327.750000,7927.666667,7213.458333,24.0,18835.750000,394.791667,1444.125000,4964.666667,4717.583333,288.585936
2021-02-25 08:00:00+00:00,13912.625000,7923.000000,7296.708333,24.0,17610.916667,394.708333,1443.583333,11371.083333,4720.916667,266.183204
2021-02-25 09:00:00+00:00,12796.458333,7907.166667,7072.875000,24.0,15405.666667,394.541667,1441.958333,17327.541667,4739.916667,244.662195
2021-02-25 10:00:00+00:00,11941.541667,7887.958333,6757.000000,24.0,14198.916667,394.500000,1448.083333,21235.916667,4754.500000,228.912385
2021-02-25 11:00:00+00:00,11075.833333,7873.625000,6386.208333,24.0,14293.958333,394.416667,1448.708333,22786.541667,4734.041667,215.616304


---

### More thorough evaluation

In [65]:
# Number of days to test
NUM_TEST_DAYS = 7
TESTING_MODELS = [CarbonIntensityForecastModels.SEASONAL_NAIVE,
                 CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING,
                 CarbonIntensityForecastModels.ARIMA,
                 CarbonIntensityForecastModels.PROPHET,
                 CarbonIntensityForecastModels.PERIODIC_PERSISTENCE]

In [66]:
# Initialise tracking variables
error = {}
runtime = {}
for model in TESTING_MODELS:
    error[model] = 0
    runtime[model] = 0

In [67]:
# Initialise first training data
train_end = data.index[-1] - timedelta(days=NUM_TEST_DAYS)
train = data.loc[:train_end, :]

In [68]:
# Loop through all days in testing
for day in range(0, NUM_TEST_DAYS):
    
    print(day+1, "of", NUM_TEST_DAYS)
    
    # Create test data
    test_start = train.index[-1] + timedelta(hours=1)
    test_end = test_start + timedelta(days=1) - timedelta(hours=1)
    test = data.loc[test_start:test_end, :]
    
    # Loop through models
    for model in TESTING_MODELS:
        
        print(" -", model)
    
        # Train model & update tracking variables
        start_time = time.time()
        forecast = generate_forecast(train, model)
        runtime[model] = runtime[model] + float(time.time()-start_time)
        error[model] = error[model] + calculate_error(train['carbon_intensity'], 
                                             test['carbon_intensity'], 
                                             forecast['carbon_intensity'])
    
    # Update train data set
    train = train.append(test)

1 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.PROPHET
 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
2 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.PROPHET
 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
3 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING


...

 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.PROPHET
 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
6 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.PROPHET
 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE
7 of 7
 - CarbonIntensityForecastModels.SEASONAL_NAIVE
 - CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING


/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/Users/juliandehoog/.pyenv/versions/3.8.7/lib/python3.8/site-packages/statsmodels/tsa/holtwinters/model.py:920: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



 - CarbonIntensityForecastModels.ARIMA
 - CarbonIntensityForecastModels.PROPHET
 - CarbonIntensityForecastModels.PERIODIC_PERSISTENCE


In [69]:
print(error)
# Also write to file
pkl_path = os.path.join('data/error_mix.pickle')
with open(pkl_path, 'wb') as file:
    pickle.dump(data, file)

{<CarbonIntensityForecastModels.SEASONAL_NAIVE: 'seasonal_naive'>: 6.481352901620877, <CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING: 'exponential_smoothing'>: 32.06842442479697, <CarbonIntensityForecastModels.ARIMA: 'arima'>: 7.408450552330471, <CarbonIntensityForecastModels.PROPHET: 'prophet'>: 5.480133822889471, <CarbonIntensityForecastModels.PERIODIC_PERSISTENCE: 'periodic_persistence'>: 5.440899046856311}


In [70]:
print(runtime)
# Also write to file
pkl_path = os.path.join('data/runtime_mix.pickle')
with open(pkl_path, 'wb') as file:
    pickle.dump(data, file)

{<CarbonIntensityForecastModels.SEASONAL_NAIVE: 'seasonal_naive'>: 0.8637294769287109, <CarbonIntensityForecastModels.EXPONENTIAL_SMOOTHING: 'exponential_smoothing'>: 17.546770334243774, <CarbonIntensityForecastModels.ARIMA: 'arima'>: 22.152767658233643, <CarbonIntensityForecastModels.PROPHET: 'prophet'>: 680.9582667350769, <CarbonIntensityForecastModels.PERIODIC_PERSISTENCE: 'periodic_persistence'>: 0.20601320266723633}


In [71]:
error2 = {}
for m in error:
    error2[m.value] = error[m]/NUM_TEST_DAYS
error2

{'seasonal_naive': 0.925907557374411,
 'exponential_smoothing': 4.58120348925671,
 'arima': 1.058350078904353,
 'prophet': 0.7828762604127816,
 'periodic_persistence': 0.7772712924080444}

In [72]:
runtime2 = {}
for m in error:
    runtime2[m.value] = runtime[m]/NUM_TEST_DAYS
runtime2

{'seasonal_naive': 0.12338992527553014,
 'exponential_smoothing': 2.5066814763205394,
 'arima': 3.1646810940333774,
 'prophet': 97.27975239072528,
 'periodic_persistence': 0.029430457523890903}

In [73]:
list(error2.keys())

['seasonal_naive',
 'exponential_smoothing',
 'arima',
 'prophet',
 'periodic_persistence']

In [74]:
trace = go.Scatter(
    x = list(error2.values()),
    y = list(runtime2.values()),
    mode = 'markers+text',
    text = list(error2.keys()),
    textposition = "bottom right",
)
fig = go.Figure(data=[trace])
fig.update_xaxes(title="Average error (MASE)", type="log")
fig.update_yaxes(title="Average running time (seconds)", type="log")
iplot(fig)

---

## 4. More involved models taking into account weather etc.